# Sweep parameters of tilted field simulations

## Frequency Sweeps

To find the critical frequency, I will perform sweeps of the frequency, at constant values of tilt angle $\delta$. After this, I will calculate the order parameters:
* nearest neighbor distance
* average angular velocity

from these, I will obtain the critical frequency and the phase diagram. 
I will do this for several heights.

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st
import support as sp

from tqdm import tqdm_notebook

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
directory = "/home/aortiza/Data/Dimers/FrequencySweep_3.9/"

In [3]:
h = 3.9*ureg.um
rho = 1.4 * ureg.um

region, positions = mgc.ordered_dimers(
    n_of_particles = 250,
    packing=0.35, height = h.magnitude,
    r = rho.magnitude, theta = np.pi/2)

particles = mgc.particles(
    positions*ureg.um,
    radius = 1.4*ureg.um,
    susceptibility = 0.4,
    diffusion=0.073*ureg.um**2/ureg.s,
    density = 1000*ureg.kg/ureg.m**3,
    temperature=300*ureg.K)

In [4]:
fmax = 7.5*ureg.Hz # 7.5Hz
dt = 15*ureg.sec # 15sec
df = 0.125*ureg.Hz

ratio = df/dt
total_time = fmax/ratio

f = st.Template("$ratio*ceil(time/$dt)*$dt").substitute(
    dt = dt.to(ureg.us).magnitude,
    ratio = ratio.to((ureg.MHz/ureg.us)).magnitude) * ureg.MHz

frequency = lambda time: (ratio.to((ureg.MHz/ureg.us))*(np.ceil(time.to(ureg.us)/dt.to(ureg.us))*dt)).to(ureg.Hz)
tilt_angle = np.concatenate([np.arange(0,30.5,0.5)])

In [5]:
def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

In [6]:
world = mgc.world(particles, temperature = 300*ureg.K,
                  lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)],
                  region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                  dipole_cutoff = 30*ureg.um)

In [7]:
index_name = "index.dat"

import csv

def prepare_simulation(delta):

    field = mgc.field(magnitude = 7.28*ureg.mT, frequency = f,
                  angle = 27*ureg.degrees, multibody_iter=2)
    
    field = redefine_field(field,delta*ureg.deg)

    sim = mgc.sim(dir_name = directory,
                  file_name="Tilt_%s_deg_h%u"%(delta,h.to(ureg.nm).magnitude),stamp_time=True,
                  timestep = 1e-4*ureg.s, framerate = 75*ureg.Hz, total_time = total_time,
                  particles = particles, world = world, field = field, output=["x","y","z","mux","muy","muz","fx","fy"])
    
    sim.tilt = delta
    sim.height = h
    
    return sim

In [8]:
simulations = [prepare_simulation(delta) for delta in tilt_angle[:]]

In [9]:
def run_simulation(sim):
    
    import csv
    import os
    index_name = "index.dat"
    directory = sim.dir_name
    
    sim.generate_scripts()
    sim.run()
    
    h = sim.height
    delta = sim.tilt
    
    with open(os.path.join(directory,index_name),'a',newline='') as file:
        writer = csv.writer(file,delimiter=' ')
        writer.writerow([os.path.split(sim.base_name)[-1], h.magnitude, delta])

## Run asynchronously in several cores at once.

Because this is running in linux, it can be done directly through the multiprocessing package. To run in windows see:
https://gist.github.com/aortiza/dd1eb47fa675c147bd44df3a98bc7a4e

In [10]:
from multiprocessing import Pool

In [11]:
if __name__ ==  '__main__': 
    num_processors = 36
    p=Pool(processes = num_processors)
    p.map(run_simulation,simulations)

In [12]:
index = pd.read_csv(os.path.join(directory,"index.dat"),names = ["file", "height", "tilt"], sep=" ")

## h = 3.5$\mu$m

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st
import support as sp

from tqdm import tqdm_notebook

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
directory = "/home/aortiza/Data/Dimers/FrequencySweep_3.5/"

In [3]:
h = 3.5*ureg.um
rho = 1.4 * ureg.um

region, positions = mgc.ordered_dimers(
    n_of_particles = 250,
    packing=0.35, height = h.magnitude,
    r = rho.magnitude, theta = np.pi/2)

particles = mgc.particles(
    positions*ureg.um,
    radius = 1.4*ureg.um,
    susceptibility = 0.4,
    diffusion=0.073*ureg.um**2/ureg.s,
    density = 1000*ureg.kg/ureg.m**3,
    temperature=300*ureg.K)

In [4]:
fmax = 7.5*ureg.Hz # 7.5Hz
dt = 15*ureg.sec # 15sec
df = 0.125*ureg.Hz

ratio = df/dt
total_time = fmax/ratio

f = st.Template("$ratio*ceil(time/$dt)*$dt").substitute(
    dt = dt.to(ureg.us).magnitude,
    ratio = ratio.to((ureg.MHz/ureg.us)).magnitude) * ureg.MHz

frequency = lambda time: (ratio.to((ureg.MHz/ureg.us))*(np.ceil(time.to(ureg.us)/dt.to(ureg.us))*dt)).to(ureg.Hz)
tilt_angle = np.concatenate([np.arange(0,30.5,0.5)])

In [5]:
def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

In [6]:
world = mgc.world(particles, temperature = 300*ureg.K,
                  lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)],
                  region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                  dipole_cutoff = 30*ureg.um)

In [7]:
index_name = "index.dat"

import csv

def prepare_simulation(delta):

    field = mgc.field(magnitude = 7.28*ureg.mT, frequency = f,
                  angle = 27*ureg.degrees, multibody_iter=2)
    
    field = redefine_field(field,delta*ureg.deg)

    sim = mgc.sim(dir_name = directory,
                  file_name="Tilt_%s_deg_h%u"%(delta,h.to(ureg.nm).magnitude),stamp_time=True,
                  timestep = 1e-4*ureg.s, framerate = 75*ureg.Hz, total_time = total_time,
                  particles = particles, world = world, field = field, output=["x","y","z","mux","muy","muz","fx","fy"])
    
    sim.tilt = delta
    sim.height = h
    
    return sim

In [8]:
simulations = [prepare_simulation(delta) for delta in tilt_angle[:]]

In [9]:
def run_simulation(sim):
    
    import csv
    import os
    index_name = "index.dat"
    directory = sim.dir_name
    
    sim.generate_scripts()
    sim.run()
    
    h = sim.height
    delta = sim.tilt
    
    with open(os.path.join(directory,index_name),'a',newline='') as file:
        writer = csv.writer(file,delimiter=' ')
        writer.writerow([os.path.split(sim.base_name)[-1], h.magnitude, delta])

## Run asynchronously in several cores at once.

Because this is running in linux, it can be done directly through the multiprocessing package. To run in windows see:
https://gist.github.com/aortiza/dd1eb47fa675c147bd44df3a98bc7a4e

In [10]:
from multiprocessing import Pool

In [11]:
if __name__ ==  '__main__': 
    num_processors = 36
    p=Pool(processes = num_processors)
    p.map(run_simulation,simulations)

In [12]:
index = pd.read_csv(os.path.join(directory,"index.dat"),names = ["file", "height", "tilt"], sep=" ")